In [7]:
### Instalación de librerias
!pip install dash dash-bootstrap-components pandas matplotlib seaborn

In [8]:
# Importación de librerías necesarias
import dash  # Librería principal para crear dashboards
from dash import dcc, html, dash_table  # Componentes para gráficos, diseño y tablas en Dash
from dash.dependencies import Input, Output, State  # Manejo de interacciones en Dash
import pandas as pd  # Para manipulación y análisis de datos
import base64  # Para codificar y decodificar datos en Base64
import io  # Para manejar contenido de archivos cargados en la aplicación
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
from collections import Counter
import re
from wordcloud import WordCloud
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
import joblib
import requests
from matplotlib.ticker import FuncFormatter
import matplotlib
import matplotlib.font_manager as fm

In [9]:
# Configuración inicial de la aplicación Dash
app = dash.Dash(__name__)  # Crear la instancia de la aplicación
app.title = "Dashboard de Fallas"  # Título que aparecerá en la pestaña del navegador

In [10]:
# Estilo general para las pestañas del dashboard
tabs_styles = {
    'backgroundColor': '#003064',  # Color de fondo de las pestañas
    'color': '#ff8300',  # Color del texto en las pestañas
    "font-family": "'Poppins', sans-serif",
    'fontWeight': 'bold',  # Negrita para el texto
    'padding': '10px'  # Espaciado interno en las pestañas
}

# Función para leer archivos con múltiples codificaciones
def leer_archivo_seguro(data):
    # Lista de codificaciones comunes que se intentarán
    codificaciones = ['utf-8', 'latin1', 'ISO-8859-1', 'gb2312']
    for codificacion in codificaciones:
        try:
            # Intentar leer el archivo con la codificación actual
            return pd.read_csv(io.StringIO(data.decode(codificacion)))
        except Exception:
            # Si falla, pasar a la siguiente codificación
            continue
    # Si todas las codificaciones fallan, usar reemplazo forzado
    return pd.read_csv(io.StringIO(data.decode('utf-8', errors='replace')))

# Función para descargar y codificar imágenes en formato Base64
def encode_image_from_url(image_url):
    response = requests.get(image_url)  # Solicitar la imagen desde la URL
    if response.status_code == 200:  # Si la solicitud es exitosa
        return base64.b64encode(response.content).decode()  # Codificar la imagen en Base64
    else:  # Si la solicitud falla
        raise Exception(f"Error al descargar la imagen desde {image_url}")

# URLs de las imágenes que se mostrarán en el dashboard
logo_forjas_url = "https://raw.githubusercontent.com/ivandiaz25/Proyecto-FB/master/3%20Entrega/bases%20de%20datos%20a%20cargar/Logo%20FB.png"
logo_udea_url = "https://raw.githubusercontent.com/ivandiaz25/Proyecto-FB/master/3%20Entrega/bases%20de%20datos%20a%20cargar/Escudo-UdeA.svg.png"

# Codificar las imágenes para que puedan mostrarse en el dashboard
logo_forjas_base64 = encode_image_from_url(logo_forjas_url)
logo_udea_base64 = encode_image_from_url(logo_udea_url)

# Verificar la codificación de las imágenes (opcional)
print("Logo Forjas codificado:", logo_forjas_base64[:50], "...")  # Mostrar primeros 50 caracteres
print("Logo UdeA codificado:", logo_udea_base64[:50], "...")  # Mostrar primeros 50 caracteres

def obtener_base_de_datos():
    # Simula cargar una base de datos desde una fuente interna o archivo local
    data = {
        'NombProducto': ['Producto1', 'Producto2'],
        'Empresa': ['Empresa1', 'Empresa2'],
        'Planta': ['Planta1', 'Planta2'],
        'País': ['País1', 'País2'],
        'Ciudad': ['Ciudad1', 'Ciudad2'],
        'Tipo de equipo': ['Equipo1', 'Equipo2'],
        'Material': ['Material1', 'Material2'],
        'Causa de desgaste': ['Desgaste1', 'Desgaste2'],
        'Si escogió otros, ¿cuál?': ['Otro1', 'Otro2'],
        'Escoja el desgaste donde se presenta': ['Desgaste1', 'Desgaste2'],
        'Tipo de cadena': ['Cadena1', 'Cadena2'],
        'Referencia cadena': ['Ref1', 'Ref2'],
        'No conformidad': ['NC1', 'NC2'],
        'Tiempo hasta falla (Dias)': [100, 200],
        'Porcentaje de elongación máx % (CADENA)': [10, 20]
    }
    return pd.DataFrame(data)

def procesar_bd_fallas(BD_fallas):
    """
    Procesa la base de datos `BD_fallas` para corregir nombres de columnas,
    eliminar duplicados, y realizar transformaciones necesarias.

    Args:
        BD_fallas (pd.DataFrame): DataFrame original a procesar.

    Returns:
        pd.DataFrame: DataFrame procesado.
    """
    # Crear una copia del DataFrame para no modificar el original
    BD_fallas = BD_fallas.copy()

    # Eliminar duplicados
    BD_fallas = BD_fallas.drop_duplicates(subset=["ConsecPedido", "NombProducto"])

    # Diccionario para corregir los nombres de las columnas
    correcciones = {
        'PaÃ­s': 'País',
        'Si escogiÃ³ otros, cuÃ¡l?': 'Si escogió otros, ¿cuál?',
        'Escoja el desgaste donde se presenta': 'Escoja el desgaste donde se presenta',
        'Porcentaje de elongaciÃ³n actual % (CADENA)': 'Porcentaje de elongación actual % (CADENA)',
        'Porcentaje de elongaciÃ³n mÃ¡x % (CADENA)': 'Porcentaje de elongación máx % (CADENA)',
        'ProyecciÃ³n en horas de operaciÃ³n restantes (H) (CADENA)': 'Proyección en horas de operación restantes (H) (CADENA)',
        'Medida sin desgaste en mm (BUJE)': 'Medida sin desgaste en mm (BUJE)',
        'Medida actual en mm1 (BUJE)': 'Medida actual en mm1 (BUJE)',
        'Porcentaje de desgaste actual % (BUJE)': 'Porcentaje de desgaste actual % (BUJE)',
        'Medida sin desgaste en mm1 (PASADOR)': 'Medida sin desgaste en mm1 (PASADOR)',
        'Medida actual en mm (PASADOR)': 'Medida actual en mm (PASADOR)',
        'Porcentaje de desgaste actual % (PASADOR)': 'Porcentaje de desgaste actual % (PASADOR)',
        'Medida sin desgaste en mm2 (RODILLO)': 'Medida sin desgaste en mm2 (RODILLO)',
        'Medida actual en mm2 (RODILLO)': 'Medida actual en mm2 (RODILLO)',
        'Porcentaje de desgaste actual % (RODILLO)': 'Porcentaje de desgaste actual % (RODILLO)',
        'Altura de la platina en mm': 'Altura de la platina en mm',
        'Medida actual de la platina en mm (PLATINA)': 'Medida actual de la platina en mm (PLATINA)',
        'Porcentaje de desgaste actual % (PLATINA)': 'Porcentaje de desgaste actual % (PLATINA)',
        'Paso en mm': 'Paso en mm',
        'Cantidad de pasos': 'Cantidad de pasos'
    }

    # Aplicar las correcciones de nombres de columnas
    BD_fallas.rename(columns=correcciones, inplace=True)

    # Correcciones para la columna 'Causa de desgaste'
    correcciones_causa = {
        'ElongaciÃ³n prematura;DesalineaciÃ³n;Otros': 'Elongación prematura;Desalineación;Otros',
        'ElongaciÃ³n prematura;CavitaciÃ³n seca': 'Elongación prematura;Cavitación seca',
        'DesalineaciÃ³n': 'Desalineación',
        'ElongaciÃ³n prematura;Otros': 'Elongación prematura;Otros',
        'CavitaciÃ³n seca': 'Cavitación seca',
        'Otros': 'Otros',
        'DesalineaciÃ³n;ElongaciÃ³n prematura;Otros;Falta de lubricaciÃ³n;Ruptura': 'Desalineación;Elongación prematura;Otros;Falta de lubricación;Ruptura',
        'Velocidad incorrecta;DesalineaciÃ³n': 'Velocidad incorrecta;Desalineación',
        'DesalineaciÃ³n;CavitaciÃ³n seca;Otros': 'Desalineación;Cavitación seca;Otros',
        'Medidas fuera de especificaciÃ³n': 'Medidas fuera de especificación',
        'Ruptura': 'Ruptura',
        'ElongaciÃ³n prematura': 'Elongación prematura',
        'Falta de lubricaciÃ³n': 'Falta de lubricación'
    }
    BD_fallas['Causa de desgaste'] = BD_fallas['Causa de desgaste'].replace(correcciones_causa)

    # Correcciones para la columna 'Material'
    BD_fallas['Material'] = BD_fallas['Material'].replace({
        'CAÃ\x91A': 'CAÑA',
        'CARBÃ\x93N GRANULADO': 'CARBÓN GRANULADO',
        pd.NA: 'DESCONOCIDO'
    })

    # Correcciones para la columna 'País'
    BD_fallas['País'] = BD_fallas['País'].replace({
        'PANAMÃ\x81': 'PANAMA'
    })

    # Cambiar formato de fechas y crear columna 'Tiempo hasta falla'
    BD_fallas['FechEstInicil'] = pd.to_datetime(BD_fallas['FechEstInicil'], errors='coerce')
    BD_fallas['Fecha de visita'] = pd.to_datetime(BD_fallas['Fecha de visita'], errors='coerce')
    BD_fallas['Tiempo hasta falla (Dias)'] = (BD_fallas['Fecha de visita'] - BD_fallas['FechEstInicil']).dt.days

    # Rellenar valores nulos con 'No Aplica'
    BD_fallas = BD_fallas.fillna('No Aplica')

    # Función personalizada para revisar y reemplazar valores de 'NombProducto'
    def reemplazar_producto(nombre):
        if 'PTNA' in nombre:
            return 'PLATINA'
        elif 'BUJE' in nombre:
            return 'BUJE'
        elif 'CDNA' in nombre:
            return 'CADENA'
        elif 'PASADOR' in nombre:
            return 'PASADOR'
        elif 'RODILLO' in nombre:
            return 'RODILLO'
        else:
            return nombre

    # Aplicar la función personalizada a la columna 'NombProducto'
    BD_fallas['NombProducto'] = BD_fallas['NombProducto'].apply(reemplazar_producto)

    return BD_fallas



Logo Forjas codificado: iVBORw0KGgoAAAANSUhEUgAABH0AAAFgCAYAAAAmZgVWAAAACX ...
Logo UdeA codificado: iVBORw0KGgoAAAANSUhEUgAAAMoAAAD7CAYAAAArQwugAAAABG ...


In [11]:
# Estilos de las pestañas
tabs_styles = {
    'backgroundColor': '#003064',
    'color': '#ff8300',
    "font-family": "'Poppins', sans-serif",
    'fontWeight': 'bold',
    'padding': '10px',
}

tabs_selected_style = {
    'backgroundColor': '#FFFFFF',
    'color': '#ff8300',
    "font-family": "'Poppins', sans-serif",
    'fontWeight': 'bold',
    'padding': '10px'
}

# Diseño del dashboard (layout)
app.layout = html.Div([
    html.Link(
        href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;500;600;700&display=swap",
        rel="stylesheet"
    ),
        # Estilo global
    # Contenedor principal del dashboard
    html.Div([
        # Componente de pestañas (Tabs)
        dcc.Tabs(
            id='tabs',  # ID del componente Tabs
            value='info-tab',  # Pestaña inicial que se muestra al cargar el dashboard
            vertical=True,  # Configura las pestañas para que se muestren verticalmente
            children=[
                # Pestaña 1: Información
                dcc.Tab(label="Información", value='info-tab', children=[
                    html.Div([
                        # Encabezado con el logo y el título
                        html.Div([
                            html.Img(
                                src='data:image/png;base64,{}'.format(logo_forjas_base64),  # Logo Forjas en Base64
                                style={'height': '50px', 'marginRight': '10px'}  # Estilo del logo
                            ),
                            html.H1(
                                "Desarrollo de modelos de analítica descriptiva y predictiva como herramienta para gobierno de datos claves en la prevención y análisis de falla",  # Título del proyecto
                                style={'color': '#ff8300',"font-family": "'Poppins', sans-serif",'textAlign':'justify', 'display': 'inline-block', 'margin': '0 20px'}  # Estilo
                            ),
                            html.Img(
                                src='data:image/png;base64,{}'.format(logo_udea_base64),  # Logo UdeA en Base64
                                style={'height': '120px', 'marginLeft': '10px'}  # Estilo del logo aumentado
                            ),
                        ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'}),  # Estilo general
                        #html.P("Este dashboard es el resultado final del proyecto:"),  # Descripción del proyecto
                        html.P("Este dashboard implementa modelos de analítica descriptiva y predictiva para apoyar la visualización de las variables relacionadas con el porcentaje de no conformidad que corresponden al porcentaje de elongación y al porcentaje de desgaste, de las cadenas forjadas. Estos modelos son generados a partir de las bases de datos generadas a partir de los servicios técnicos y de la planta virtual. Estos modelos apoyan la toma de decisiones en el área de la confiabilidad de los productos analizados."),
                        html.P("El dashboard presenta tres módulos. El primer módulo, llamado Datos, permite cargar las bases de datos. El segundo módulo, llamado Gráficos se subdivide en tres pestañas: 1). Análisis descriptivo de las variables explicativas, 2). Análisis del porcentaje de no conformidad vs una variable explicativa y, 3). Análisis del porcentaje de no conformidad vs dos variables explicativas. Por último, el tercer módulo, llamado Predicción, presenta un modelo predictivo basado en técnicas de aprendizaje automático que predice el porcentaje de desgaste y de elongación. El modelo se evalúa mediante métricas de desempeño como error cuadrático medio (MSE), error absoluto medio (MAE) y error porcentual absoluto medio (MAPE)."),
                    ], style={'textAlign':'justify',"font-family": "'Poppins', sans-serif", 'padding': '20px'})  # Estilo del contenido
                ], style=tabs_styles, selected_style=tabs_selected_style),  # Estilo de la pestaña
                # Pestaña 2: Datos
                dcc.Tab(label="Datos", value='data-tab', children=[
                    html.Div([  # Contenido de la pestaña de datos
                        html.H2("Cargar Bases de Datos", style={'color': '#ff8300',"font-family": "'Poppins', sans-serif"}),  # Encabezado
                        html.Div([  # Contenedor para los botones de carga
                            dcc.Upload(  # Componente para cargar el primer archivo
                                id='upload-data-1',  # ID del componente
                                children=html.Button(
                                'Cargar BD_Informe Servicio',
                                style={"font-family": "'Poppins', sans-serif"}  # Aplicar estilo directamente al texto del botón
                            ),  # Botón de carga
                            style={'margin': '10px'}  # Espaciado alrededor del botón
                        ),
                            dcc.Upload(  # Componente para cargar el segundo archivo
                                id='upload-data-2',  # ID del componente
                                children=html.Button(
                                    'Cargar BD_Planta Virtual',
                                    style={"font-family": "'Poppins', sans-serif"}  # Aplicar estilo directamente al texto del botón
                                ),  # Botón de carga
                                style={'margin': '10px'}  # Espaciado alrededor del botón
                            ),
                        ], style={'display': 'flex'}), # Alinear botones horizontalmente
                        html.Div(id='output-data-upload', style={'marginTop': '20px',"font-family": "'Poppins', sans-serif"}),  # Contenedor para mostrar resultados
                    ])
                ], style=tabs_styles, selected_style=tabs_selected_style),  # Estilo de la pestaña

                # Nueva pestaña: Gráficos
                dcc.Tab(label="Gráficos", value='charts-tab', children=[
                    dcc.Tabs(
                        id='charts-subtabs',
                        value='quantitative-tab',
                        vertical=False,
                        children=[
                            # Subpestaña 1: Análisis cuantitativo de No conformidades
                            dcc.Tab(label="Análisis descriptivo de las variables explicativas", value='quantitative-tab', children=[
                                html.Div([
                                    html.H3("Seleccione una variable:"),
                                    dcc.Dropdown(
                                        id='quantitative-menu',
                                        options=[
                                            {'label': 'Causa de desgaste', 'value': 'bar_cause'},
                                            {'label': 'Materiales', 'value': 'pie_materials'},
                                            {'label': 'Distribución de No conformidad', 'value': 'pie_fallas'},
                                            {'label': 'No conformidad por Región', 'value': 'pie_region'},
                                            {'label': 'Distribución de Tipos de Equipo', 'value': 'pie_equipment'},
                                            {'label': 'Distribución de Causa de Desgaste', 'value': 'pie_cause'},
                                            {'label': 'Histograma no conformidad', 'value': 'histogram_fallas'}
                                        ],
                                        value='bar_cause',  # Selección inicial
                                        style={'marginBottom': '20px',"font-family": "'Poppins', sans-serif",'textAlign':'center'}
                                    ),
                                    html.Div(id='quantitative-content')  # Contenedor dinámico para mostrar los gráficos
                                ], style={'padding': '20px',"font-family": "'Poppins', sans-serif",'textAlign':'center'})
                            ], style=tabs_styles, selected_style=tabs_selected_style),

                            # Subpestaña 2: Análisis gráfico de variables explicativas
                            dcc.Tab(label="Análisis del porcentaje de no conformidad vs una variable explicativa", value='explanatory-tab', children=[
                                html.Div([
                                    html.H3("Seleccione una variable para analizar con el porcentaje de no conformidad:"),
                                    dcc.Dropdown(
                                        id='explanatory-dropdown',
                                        options=[{'label': var, 'value': var} for var in [
                                            'Actividad', 'RecMAQ', 'RecHOM', 'Empresa', 'Planta', 'País',
                                            'Ciudad', 'Tipo de equipo', 'Material', 'Causa de desgaste',
                                            'Escoja el desgaste donde se presenta', 'Tipo de cadena', 'Referencia cadena'
                                        ]],
                                        value='Actividad',  # Valor por defecto
                                        style={'marginBottom': '20px',"font-family": "'Poppins', sans-serif",'textAlign':'center'}
                                    ),
                                    html.Div(id='explanatory-graph')  # Contenedor para el gráfico dinámico
                                ], style={'padding': '20px',"font-family": "'Poppins', sans-serif",'textAlign':'center'})
                            ], style=tabs_styles, selected_style=tabs_selected_style),

                            # Subpestaña 3: Análisis gráfico de variables explicativas bivariadas
                            dcc.Tab(label="Análisis del porcentaje de no conformidad vs dos variables explicativas", value='bivariate-tab', children=[
                                html.Div([
                                    html.H3("Seleccione una variable para analizar con el porcentaje de no conformidad y el tipo de 'No conformidad':"),
                                    dcc.Dropdown(
                                        id='bivariate-dropdown',
                                        options=[{'label': var, 'value': var} for var in [
                                            'Actividad', 'RecMAQ', 'RecHOM', 'Empresa', 'Planta', 'País',
                                            'Ciudad', 'Tipo de equipo', 'Material', 'Causa de desgaste',
                                            'Escoja el desgaste donde se presenta', 'Tipo de cadena', 'Referencia cadena'
                                        ]],
                                        value='Actividad',  # Selección inicial
                                        style={'marginBottom': '20px',"font-family": "'Poppins', sans-serif",'textAlign':'center'}
                                    ),
                                    html.Div(id='bivariate-graph')  # Contenedor dinámico para gráficos
                                ], style={'padding': '20px',"font-family": "'Poppins', sans-serif",'textAlign':'center'})
                            ], style=tabs_styles, selected_style=tabs_selected_style)
                        ]
                    )
                ], style=tabs_styles, selected_style=tabs_selected_style),

                # Pestaña 4: Modelo
                dcc.Tab(label="Predicción", value='model-tab', children=[
                    html.Div([
                        html.H1(
                            "Predicción del Porcentaje de No Conformidad",
                            style={
                                'color': '#FF8C00',  # Color naranja
                                'textAlign': 'center',  # Centrado
                                "font-family": "'Poppins', sans-serif",'textAlign':'center'
                            }
                        ),
                        html.Div(id='layout-dinamico')
                    ])
                ], style=tabs_styles, selected_style=tabs_selected_style)

            ]
        )
    ])
])


In [12]:
# Callback para procesar y combinar datos
@app.callback(
    Output('output-data-upload', 'children'),  # Salida: Contenedor para mostrar resultados
    [Input('upload-data-1', 'contents'),  # Entrada: Contenido del primer archivo cargado
     Input('upload-data-2', 'contents')],  # Entrada: Contenido del segundo archivo cargado
    [State('upload-data-1', 'filename'),  # Estado: Nombre del primer archivo cargado
     State('upload-data-2', 'filename')]  # Estado: Nombre del segundo archivo cargado
)
def update_output(contents1, contents2, filename1, filename2):
    # Función que procesa los datos
    # Validar que ambos archivos han sido cargados
    if contents1 and contents2:
        try:
            # Procesar archivo 1
            content_type1, content_string1 = contents1.split(',')  # Dividir el contenido para decodificar
            decoded1 = base64.b64decode(content_string1)  # Decodificar contenido Base64
            df_informe = leer_archivo_seguro(decoded1)  # Leer el archivo como DataFrame

            # Procesar archivo 2
            content_type2, content_string2 = contents2.split(',')  # Dividir el contenido para decodificar
            decoded2 = base64.b64decode(content_string2)  # Decodificar contenido Base64
            df_planta = leer_archivo_seguro(decoded2)  # Leer el archivo como DataFrame

            # Mostrar una vista previa de los datos cargados
            preview_informe = df_informe.head().to_dict('records')  # Obtener las primeras filas del DataFrame 1
            preview_planta = df_planta.head().to_dict('records')  # Obtener las primeras filas del DataFrame 2

            # Unir bases de datos
            BD_fallas = pd.merge(
                df_planta,  # Primer DataFrame
                df_informe,  # Segundo DataFrame
                left_on="ConsecPedido",  # Columna de unión en el primer DataFrame
                right_on="Pedido",  # Columna de unión en el segundo DataFrame
                how="inner"  # Tipo de unión: solo filas coincidentes
            )

            # Eliminar duplicados
            BD_fallas = BD_fallas.drop_duplicates(subset=["ConsecPedido", "NombProducto"])  # Eliminar duplicados por columnas clave

            # Crear un diccionario para corregir los nombres de las columnas

            correcciones = {
                'PaÃ­s': 'País',
                'Si escogiÃ³ otros, cuÃ¡l?': 'Si escogió otros, ¿cuál?',
                'Escoja el desgaste donde se presenta': 'Escoja el desgaste donde se presenta',
                'Porcentaje de elongaciÃ³n actual % (CADENA)': 'Porcentaje de elongación actual % (CADENA)',
                'Porcentaje de elongaciÃ³n mÃ¡x % (CADENA)': 'Porcentaje de elongación máx % (CADENA)',
                'ProyecciÃ³n en horas de operaciÃ³n restantes (H) (CADENA)': 'Proyección en horas de operación restantes (H) (CADENA)',
                'Medida sin desgaste en mm (BUJE)': 'Medida sin desgaste en mm (BUJE)',
                'Medida actual en mm1 (BUJE)': 'Medida actual en mm1 (BUJE)',
                'Porcentaje de desgaste actual % (BUJE)': 'Porcentaje de desgaste actual % (BUJE)',
                'Medida sin desgaste en mm1 (PASADOR)': 'Medida sin desgaste en mm1 (PASADOR)',
                'Medida actual en mm (PASADOR)': 'Medida actual en mm (PASADOR)',
                'Porcentaje de desgaste actual % (PASADOR)': 'Porcentaje de desgaste actual % (PASADOR)',
                'Medida sin desgaste en mm2 (RODILLO)': 'Medida sin desgaste en mm2 (RODILLO)',
                'Medida actual en mm2 (RODILLO)': 'Medida actual en mm2 (RODILLO)',
                'Porcentaje de desgaste actual % (RODILLO)': 'Porcentaje de desgaste actual % (RODILLO)',
                'Altura de la platina en mm': 'Altura de la platina en mm',
                'Medida actual de la platina en mm (PLATINA)': 'Medida actual de la platina en mm (PLATINA)',
                'Porcentaje de desgaste actual % (PLATINA)': 'Porcentaje de desgaste actual % (PLATINA)',
                'Paso en mm': 'Paso en mm',
                'Cantidad de pasos': 'Cantidad de pasos'
            }

            # Aplicar las correcciones al DataFrame
            BD_fallas.rename(columns=correcciones, inplace=True)

            # Corregir el nombre de los resultados de la variable 'Causa de desgaste'
            correcciones_causa = {
                'ElongaciÃ³n prematura;DesalineaciÃ³n;Otros': 'Elongación prematura;Desalineación;Otros',
                'ElongaciÃ³n prematura;CavitaciÃ³n seca': 'Elongación prematura;Cavitación seca',
                'DesalineaciÃ³n': 'Desalineación',
                'ElongaciÃ³n prematura;Otros': 'Elongación prematura;Otros',
                'CavitaciÃ³n seca': 'Cavitación seca',
                'Otros': 'Otros',
                'DesalineaciÃ³n;ElongaciÃ³n prematura;Otros;Falta de lubricaciÃ³n;Ruptura': 'Desalineación;Elongación prematura;Otros;Falta de lubricación;Ruptura',
                'Velocidad incorrecta;DesalineaciÃ³n': 'Velocidad incorrecta;Desalineación',
                'DesalineaciÃ³n;CavitaciÃ³n seca;Otros': 'Desalineación;Cavitación seca;Otros',
                'Medidas fuera de especificaciÃ³n': 'Medidas fuera de especificación',
                'Ruptura': 'Ruptura',
                'ElongaciÃ³n prematura': 'Elongación prematura',
                'Falta de lubricaciÃ³n': 'Falta de lubricación'
            }

            BD_fallas['Causa de desgaste'] = BD_fallas['Causa de desgaste'].replace(correcciones_causa)

            # Corregir los valores en la columna 'Material'
            BD_fallas['Material'] = BD_fallas['Material'].replace({
                'CAÃ\x91A': 'CAÑA',
                'CARBÃ\x93N GRANULADO': 'CARBÓN GRANULADO',
                pd.NA: 'DESCONOCIDO'
            })

            # Corregir los valores en la columna 'País'
            BD_fallas['País'] = BD_fallas['País'].replace({
                'PANAMÃ\x81': 'PANAMA'
            })

            # Cambio de formato de fecha
            BD_fallas['FechEstInicil'] = pd.to_datetime(BD_fallas['FechEstInicil'], errors='coerce')
            BD_fallas['Fecha de visita'] = pd.to_datetime(BD_fallas['Fecha de visita'], errors='coerce')

            # Crear una nueva columna 'Tiempo hasta falla'
            BD_fallas['Tiempo hasta falla (Dias)'] = (BD_fallas['Fecha de visita'] - BD_fallas['FechEstInicil']).dt.days

            # Rellenar los valores nulos con la cadena 'No Aplica'
            BD_fallas = BD_fallas.fillna('No Aplica')

            # Función personalizada para revisar y reemplazar los valores de la columna 'NombProducto'
            def reemplazar_producto(nombre):
                if 'PTNA' in nombre:
                    return 'PLATINA'
                elif 'BUJE' in nombre:
                    return 'BUJE'
                elif 'CDNA' in nombre:
                    return 'CADENA'
                elif 'PASADOR' in nombre:
                    return 'PASADOR'
                elif 'RODILLO' in nombre:
                    return 'RODILLO'
                else:
                    return nombre

            BD_fallas['NombProducto'] = BD_fallas['NombProducto'].apply(reemplazar_producto)


            # Retornar una vista previa del resultado al usuario
            return html.Div([
                html.H3("Vista previa de BD_Informe Servicio:"),  # Encabezado para el primer archivo
                dash_table.DataTable(  # Tabla interactiva para el primer archivo
                    data=preview_informe,  # Datos del DataFrame
                    columns=[{"name": i, "id": i} for i in df_informe.columns]  # Columnas del DataFrame
                ),
                html.H3("Vista previa de BD_Planta Virtual:"),  # Encabezado para el segundo archivo
                dash_table.DataTable(  # Tabla interactiva para el segundo archivo
                    data=preview_planta,  # Datos del DataFrame
                    columns=[{"name": i, "id": i} for i in df_planta.columns]  # Columnas del DataFrame
                ),
                html.H3(f"Datos combinados: {BD_fallas.shape[0]} filas y {BD_fallas.shape[1]} columnas."),  # Resumen del DataFrame combinado
                dash_table.DataTable(  # Tabla interactiva para el DataFrame combinado
                    data=BD_fallas.head().to_dict('records'),  # Mostrar las primeras filas del DataFrame combinado
                    columns=[{"name": i, "id": i} for i in BD_fallas.columns]  # Columnas del DataFrame combinado
                )
            ])
        except Exception as e:  # Capturar errores
            # Mostrar el error al usuario
            return html.Div([html.P(f"Error al procesar los archivos: {e}", style={'color': 'red'})])
    else:
        # Si no se cargaron ambos archivos
        return html.Div("Sube ambos archivos para continuar.", style={'color': 'red'})


@app.callback(
    Output('quantitative-content', 'children'),  # Contenedor dinámico
    Input('quantitative-menu', 'value'),         # Selección del menú desplegable
    [Input('upload-data-1', 'contents'),         # Datos cargados (archivo 1)
     Input('upload-data-2', 'contents')]         # Datos cargados (archivo 2)
)
def actualizar_contenido_quantitative(selection, contents1, contents2):
    """

    Args:
      selection:
      contents1:
      contents2:

    Returns:

    """
    if contents1 and contents2:
        # Procesar las bases de datos cargadas
        content_type1, content_string1 = contents1.split(',')
        content_type2, content_string2 = contents2.split(',')
        decoded1 = base64.b64decode(content_string1)
        decoded2 = base64.b64decode(content_string2)
        df_informe = leer_archivo_seguro(decoded1)
        df_planta = leer_archivo_seguro(decoded2)

        # Combinar las bases de datos
        BD_fallas = pd.merge(
            df_planta,
            df_informe,
            left_on="ConsecPedido",
            right_on="Pedido",
            how="inner"
        )
        # Función procesar datos
        BD_fallas = procesar_bd_fallas(BD_fallas)


        if selection == 'bar_cause':
    # Gráfico de barras existente
            BD_fallas['Causa de desgaste'] = BD_fallas['Causa de desgaste'].fillna('Desconocido')
            falla_counts = BD_fallas['Causa de desgaste'].value_counts()

    # Lista de tonos anaranjados
            colores_anaranjados = ['#003063', '#003064', '#3dac2b', '#ffb600', '#4d4d4d',
                                   '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00']
    # Ajustar la cantidad de colores a la cantidad de barras
            colores_anaranjados = colores_anaranjados[:len(falla_counts)]


            fig = {
        'data': [
            {
                'x': falla_counts.index,
                'y': falla_counts.values,
                'type': 'bar',
                'marker': {'color': colores_anaranjados},
                'tickformat': '.2f',  # Mostrar decimales en el eje Y
                'tickvals': [i * 0.5 for i in range(0, int(falla_counts.values.max() * 2) + 2)],
                # Crea una lista de valores en incrementos de 0.5, hasta el valor máximo en y
                'ticktext': [f'{i:.2f}' for i in [j * 0.5 for j in range(0, int(falla_counts.values.max() * 2) + 2)]]
                # Etiquetas correspondientes para los valores del eje Y
            }
        ],
            'layout': {
            'xaxis': {'title': 'Causa de Desgaste', 'tickangle': 45,
                'titlefont': {'family': 'Poppins'}},
            'yaxis': {'title': 'Número de No Conformidades',
                'titlefont': {'family': 'Poppins'}},
            'font': {'family': 'Poppins'},  # Aplica la fuente a todo el gráfico
            'bargap': 0.2,
            'height': 500
        }
    }
            return dcc.Graph(figure=fig)

        elif selection == 'pie_materials':
    # Gráfico circular existente
            BD_fallas['Material'] = BD_fallas['Material'].fillna('Desconocido')
            material_counts = BD_fallas['Material'].value_counts()

    # Lista de tonos anaranjados
            colores_anaranjados = [ '#003063', '#003064', '#3dac2b', '#ffb600', '#4d4d4d',
                                    '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00']
    # Ajustar la cantidad de colores a la cantidad de segmentos
            colores_anaranjados = colores_anaranjados[:len(material_counts)]

            fig = {
        'data': [
            {
                'values': material_counts.values,
                'labels': material_counts.index,
                'type': 'pie',
                'hole': 0.3,
                'marker': {'colors': colores_anaranjados}
            }
        ],
        'layout': {
            #'title': 'Distribución de Tipos de Material',
            'font': {'family': 'Poppins'},
            'height': 500
        }
    }
            return dcc.Graph(figure=fig)

        elif selection == 'scatter_cause':
            if 'BD_fallas' in locals() and not BD_fallas.empty:
        # Generar el gráfico si BD_fallas está disponible
              df_jitter = BD_fallas[['Porcentaje de elongación actual % (CADENA)', 'Causa de desgaste']].copy()

            # Relación entre Elongación y Causa
            df_jitter['Porcentaje de elongación actual % (CADENA)'] = pd.to_numeric(
            df_jitter['Porcentaje de elongación actual % (CADENA)'], errors='coerce'
        )
            colores_anaranjados = [ '#003063', '#003064', '#3dac2b', '#ffb600', '#4d4d4d',
                                    '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00']
    # Ajustar la cantidad de colores a la cantidad de segmentos
            colores_anaranjados = colores_anaranjados[:len(material_counts)]

        # Eliminar valores nulos
            df_jitter = df_jitter.dropna()

            fig = {
            'data': [
                {
                    'x': df_jitter['Causa de desgaste'],
                    'y': df_jitter['Porcentaje de elongación actual % (CADENA)'],
                    'mode': 'markers',
                    'type': 'scatter',
                    'marker': {'opacity': 0.6, 'size': 8, 'color': colores_anaranjados},
                }
            ],
            'layout': {
                #'title': 'Relación entre Elongación de la Cadena y Causa de Desgaste',
                'xaxis': {'title': 'Causa de Desgaste', 'tickangle': 45, 'titlefont': {'family': 'Poppins'}},
                'yaxis': {'title': 'Porcentaje de Elongación Actual (CADENA)','titlefont': {'family': 'Poppins'}},
                'font': {'family': 'Poppins'},
                'height': 500,
                }
            }
            return dcc.Graph(figure=fig)


        elif selection == 'pie_fallas':
    # Distribución de Fallas
            falla_counts = BD_fallas['Falla'].value_counts()

            fig = {
        'data': [
            {
                'values': falla_counts.values,
                'labels': falla_counts.index,
                'type': 'pie',
                'marker': {
                    'colors': ['#003063', '#003064', '#3dac2b', '#ffb600', '#4d4d4d',
                                    '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00']  # Tonos anaranjados
                },
                'textinfo': 'label+percent',  # Mostrar etiquetas y porcentajes
            }
        ],
        'layout': {
           # 'title': 'Distribución de Fallas',
           'font': {'family': 'Poppins'},
            'height': 500
        }
    }
            return dcc.Graph(figure=fig)


        elif selection == 'boxplot_fallas':
    # Falla por Tipo de Equipo
            df_opt_CDNA = BD_fallas[['Tipo de equipo', 'Valor']].dropna()

            fig = {
        'data': [
            {
                'x': df_opt_CDNA['Tipo de equipo'],
                'y': df_opt_CDNA['Valor'],
                'type': 'box',
                'marker': {'color': '#FFA07A'},  # Tonos anaranjados
                'boxmean': True  # Mostrar la media en el boxplot
            }
        ],
        'layout': {
            #'title': 'Falla por Tipo de Equipo',
            'xaxis': {
                'title': 'Tipo de Equipo',
                'tickangle': 45, 'titlefont': {'family': 'Poppins'}
            },
            'yaxis': {
                'title': 'Porcentaje de No conformidad', 'titlefont': {'family': 'Poppins'}
            },
            'font': {'family': 'Poppins'},
            'height': 500
        }
    }
            return dcc.Graph(figure=fig)

        elif selection == 'pie_equipment':
    # Crear el gráfico circular con la distribución de tipos de equipo
            BD_fallas['Tipo de equipo'] = BD_fallas['Tipo de equipo'].fillna('Desconocido')
            falla_counts3 = BD_fallas['Tipo de equipo'].value_counts()

            fig = {
        'data': [
            {
                'values': falla_counts3.values,
                'labels': falla_counts3.index,
                'type': 'pie',
                'hole': 0,  # Para que sea un gráfico de pastel completo
                'textinfo': 'label+percent',  # Mostrar etiquetas y porcentajes
                'hoverinfo': 'label+value+percent',  # Información adicional en el hover
                'marker': {
                    'colors': ['#003063', '#003064', '#3dac2b', '#ffb600', '#4d4d4d',
                                   '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00']  # Tonos anaranjados
                }
            }
        ],
        'layout': {
          #  'title': 'Distribución de Tipos de Equipo',
          'font': {'family': 'Poppins'},
            'height': 500,
            'showlegend': True,  # Mostrar la leyenda
            'legend': {'x': 1, 'y': 1}  # Posición de la leyenda
        }
    }

            return dcc.Graph(figure=fig)
        elif selection == 'pie_cause':
    # Preprocesamiento de datos: contar frecuencias y manejar valores nulos
            BD_fallas['Causa de desgaste'] = BD_fallas['Causa de desgaste'].fillna('Desconocido')
            falla_counts = BD_fallas['Causa de desgaste'].value_counts()

    # Crear el gráfico circular
            fig = {
        'data': [
            {
                'values': falla_counts.values,  # Valores del conteo
                'labels': falla_counts.index,  # Etiquetas de las causas
                'type': 'pie',
                'hole': 0,  # Gráfico completo (sin agujero)
                'textinfo': 'label+percent',  # Mostrar etiquetas y porcentajes
                'hoverinfo': 'label+value+percent',  # Información adicional al pasar el cursor
                'marker': {
                    'colors': ['#003063', '#003064', '#3dac2b', '#ffb600', '#4d4d4d',
                                    '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00']  # Tonos anaranjados
                }
            }
        ],
        'layout': {
          #  'title': 'Distribución de Causa de Desgaste',
          'font': {'family': 'Poppins'},
            'height': 500,  # Altura del gráfico
            'showlegend': True,  # Mostrar leyenda
            'legend': {'x': 1, 'y': 1}  # Posición de la leyenda
        }
    }

            return dcc.Graph(figure=fig)
        elif selection == 'line_cumulative_fallas':
    # Asegurarse de que la columna "Fecha de visita" esté en formato datetime
            BD_fallas['Fecha de visita'] = pd.to_datetime(BD_fallas['Fecha de visita'], errors='coerce')

    # Filtrar los registros válidos que tengan fechas correctas
            BD_fallas2 = BD_fallas.dropna(subset=['Fecha de visita'])

    # Contar el número de fallas por fecha de visita
            fallas_por_fecha = BD_fallas2.groupby(BD_fallas2['Fecha de visita'].dt.date).size()

    # Calcular las fallas acumuladas
            fallas_cumuladas = fallas_por_fecha.cumsum()

    # Crear el gráfico de líneas
            fig = {
        'data': [
            {
                'x': fallas_cumuladas.index,  # Fechas
                'y': fallas_cumuladas.values,  # Fallas acumuladas
                'type': 'scatter',
                'mode': 'lines+markers',  # Líneas con marcadores
                'marker': {
                    'symbol': 'circle',
                    'size': 8,
                    'color': '#FF7F50'  # Marcadores en tono anaranjado
                },
                'line': {
                    'color': '#FF4500',  # Línea en tono anaranjado oscuro
                    'width': 2
                }
            }
        ],
        'layout': {
           # 'title': 'Fallas Acumuladas en el Tiempo',
            'xaxis': {
                'title': 'Fecha de Visita',
                'tickangle': 45, 'titlefont': {'family': 'Poppins'}
            },
            'yaxis': {
                'title': 'No conformidades Acumuladas', 'titlefont': {'family': 'Poppins'}
            },
           'font': {'family': 'Poppins'},
            'height': 500,
            'grid': {'x': True, 'y': True}  # Mostrar cuadrícula
        }
    }

            return dcc.Graph(figure=fig)

        elif selection == 'histogram_fallas':
    # Crear el histograma con datos de "Valor"
            hist_data = BD_fallas['Valor'].dropna()  # Eliminar valores nulos

    # Tonos anaranjados para las barras
            orange_shades = [
                                  '#003063', '#003064', '#3dac2b', '#ffb600', '#4d4d4d',
                                   '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00'
                                   '#9d9d9d', '#9d9d9d', '#f44611', '#252850', '#0d8e02',
                                   '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00'
    ]

    # Crear el gráfico de histograma
            fig = {
        'data': [
            {
                'x': hist_data,  # Valores a graficar
                'type': 'histogram',
                'xbins': {
                    'size': (hist_data.max() - hist_data.min()) / 30  # Dividir en 30 bins
                },
                'marker': {
                    'color': orange_shades * 2,  # Repetir colores para cubrir todas las barras
                    'line': {'width': 1, 'color': 'black'}  # Borde negro
                },
                'opacity': 0.9  # Transparencia
            }
        ],
        'layout': {
          #  'title': 'Histograma de la No Conformidad',
            'xaxis': {
                'title': 'Valor', 'titlefont': {'family': 'Poppins'}
            },
            'yaxis': {
                'title': 'Frecuencia de no Conformidad', 'titlefont': {'family': 'Poppins'}
            },
          'font': {'family': 'Poppins'},
            'height': 500,
            'bargap': 0.05,  # Espacio entre barras
            'grid': {'x': True, 'y': True}  # Mostrar cuadrícula
        }
    }

            return dcc.Graph(figure=fig)
        elif selection == 'pie_region':
    # Contar la frecuencia de cada categoría en la columna "País"
          BD_fallas['País'] = BD_fallas['País'].fillna('Desconocido')  # Manejar valores nulos
          falla_counts = BD_fallas['País'].value_counts()

    # Crear el gráfico circular
          fig = {
        'data': [
            {
                'values': falla_counts.values,  # Frecuencia de fallas
                'labels': falla_counts.index,  # Etiquetas de regiones
                'type': 'pie',
                'hole': 0,  # Gráfico completo
                'textinfo': 'label+percent',  # Mostrar etiquetas y porcentajes
                'hoverinfo': 'label+value+percent',  # Mostrar información al pasar el cursor
                'marker': {'colors': sns.color_palette("Oranges", len(falla_counts)).as_hex()}  # Colores personalizados
            }
        ],
        'layout': {
           # 'title': 'Fallas por Región',
           'font': {'family': 'Poppins'},
            'height': 500,  # Altura del gráfico
            'showlegend': True,  # Mostrar leyenda
            'legend': {'x': 1, 'y': 1}  # Posición de la leyenda
        }
    }

          return dcc.Graph(figure=fig)

          return html.Div("Cargue las bases de datos para visualizar los gráficos.", style={'color': 'red'})

@app.callback(
    Output('explanatory-graph', 'children'),
    Input('explanatory-dropdown', 'value'),
    Input('upload-data-1', 'contents'),
    Input('upload-data-2', 'contents')
)
def actualizar_grafico_explanatory(variable, contents1, contents2):
    if contents1 and contents2:
        # Procesar los archivos cargados
        content_type1, content_string1 = contents1.split(',')
        content_type2, content_string2 = contents2.split(',')
        decoded1 = base64.b64decode(content_string1)
        decoded2 = base64.b64decode(content_string2)
        df_informe = leer_archivo_seguro(decoded1)
        df_planta = leer_archivo_seguro(decoded2)

        # Combinar bases de datos
        BD_fallas = pd.merge(
            df_planta,
            df_informe,
            left_on="ConsecPedido",
            right_on="Pedido",
            how="inner"
        )

        # Procesar los datos
        BD_fallas = procesar_bd_fallas(BD_fallas)

        # Renombrar la columna 'Valor' a 'Porcentaje de no conformidad'
        BD_fallas.rename(columns={'Valor': 'Porcentaje de no conformidad'}, inplace=True)

        # Asegurarse de que los porcentajes están en la escala correcta
        if BD_fallas['Porcentaje de no conformidad'].max() <= 1:
            BD_fallas['Porcentaje de no conformidad'] *= 100


        # Verificar si la variable seleccionada está en los datos
        if variable in BD_fallas.columns:
            BD_fallas = BD_fallas[[variable, 'Porcentaje de no conformidad']].dropna().drop_duplicates()

            # Crear colores personalizados (tonos de anaranjado)
            orange_shades = [
               '#003063', '#003064', '#3dac2b', '#ffb600', '#4d4d4d',
                                    '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00'
            ]

            # Asignar colores a cada categoría en la variable seleccionada
            unique_values = BD_fallas[variable].unique()
            color_map = {value: orange_shades[i % len(orange_shades)] for i, value in enumerate(unique_values)}


            # Crear gráfico con Matplotlib y Seaborn
            plt.figure(figsize=(10, 6))
            sns.barplot(
                data=BD_fallas,
                x=variable,
                y='Porcentaje de no conformidad',
                ci=None,
                palette=[color_map[val] for val in BD_fallas[variable]]  # Aplica los colores mapeados
            )

            # Configurar el rango del eje Y
            y_min = 0  # Valor mínimo del eje Y (puedes ajustarlo si es necesario)
            y_max = BD_fallas['Porcentaje de no conformidad'].max() + 0.5  # Máximo dinámico basado en los datos
            y_ticks = np.arange(y_min, y_max + 0.5, 0.5)  # Incrementos de 0.5

            # Aplicar ticks personalizados
            plt.yticks(y_ticks, [f'{val:.2f}' for val in y_ticks])

            # Configuración de título y etiquetas
            #plt.title(f'{variable} vs Porcentaje de no conformidad', fontsize=14)
            plt.xlabel(variable, fontsize=12)
            plt.ylabel('Porcentaje de no conformidad', fontsize=12)

            # Ajustar las etiquetas del eje X
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()

            # Guardar la imagen del gráfico en un buffer
            buf = io.BytesIO()
            plt.savefig(buf, format="png")
            buf.seek(0)
            plt.close()

            # Codificar la imagen en base64 para mostrarla en el dashboard
            encoded_image = base64.b64encode(buf.read()).decode('utf-8')
            buf.close()

            # Retornar el gráfico dentro de Dash
            return html.Div([
                html.Img(
                    src=f'data:image/png;base64,{encoded_image}',
                    style={'width': '100%', 'height': 'auto'}
                )
            ], style={"font-family": "'Poppins', sans-serif"})
        else:
            return html.Div("La variable seleccionada no está disponible en los datos.")
    else:
        return html.Div("Cargue las bases de datos para generar gráficos.")


@app.callback(
    Output('bivariate-graph', 'children'),
    Input('bivariate-dropdown', 'value'),
    Input('upload-data-1', 'contents'),
    Input('upload-data-2', 'contents')
)
def actualizar_grafico_bivariado(variable, contents1, contents2):
    if contents1 and contents2:
        # Procesar archivos cargados
        content_type1, content_string1 = contents1.split(',')
        content_type2, content_string2 = contents2.split(',')
        decoded1 = base64.b64decode(content_string1)
        decoded2 = base64.b64decode(content_string2)
        df_informe = leer_archivo_seguro(decoded1)
        df_planta = leer_archivo_seguro(decoded2)

        # Combinar bases de datos
        BD_fallas = pd.merge(
            df_planta,
            df_informe,
            left_on="ConsecPedido",
            right_on="Pedido",
            how="inner"
        )

        # Procesar los datos
        BD_fallas = procesar_bd_fallas(BD_fallas)

        # Renombrar la columna 'Falla' a 'No conformidad'
        BD_fallas.rename(columns={'Falla': 'No conformidad'}, inplace=True)
        BD_fallas.rename(columns={'Valor': 'Porcentaje de no conformidad'}, inplace=True)

        # Verificar si la variable está presente en el DataFrame
        if variable in BD_fallas.columns:
            # Agrupar por 'No conformidad' y la variable seleccionada
            df_grouped = BD_fallas.groupby(['No conformidad', variable])['Porcentaje de no conformidad'].mean().reset_index()

            # Crear colores personalizados (tonos de anaranjado y azules)
            # Crear colores personalizados (tonos de anaranjado)
            orange_shades = [
               '#003063', '#003064', '#3dac2b', '#ffb600', '#4d4d4d',
                                    '#FFA500', '#3a4e7c', '#4ca23a', '#ffc400',
                                   '#606060', '#9d9d9d', '#FF4500', '#616e96', '#0d8e02', '#ffbf00'
            ]

            # Asignar colores a cada categoría en la variable seleccionada
            unique_values = BD_fallas[variable].unique()
            color_map = {value: orange_shades[i % len(orange_shades)] for i, value in enumerate(unique_values)}

            # Crear gráfico de barras con Matplotlib y Seaborn
            plt.figure(figsize=(12, 7))
            sns.barplot(
                data=df_grouped,
                x='No conformidad',
                y='Porcentaje de no conformidad',
                hue=variable,
                palette=[color_map[val] for val in unique_values]
            )

            # Configurar el rango del eje Y
            y_min = 0  # Valor mínimo del eje Y (puedes ajustarlo si es necesario)
            y_max = BD_fallas['Porcentaje de no conformidad'].max() + 0.5  # Máximo dinámico basado en los datos
            y_ticks = np.arange(y_min, y_max + 0.5, 0.5)  # Incrementos de 0.5

            # Aplicar ticks personalizados
            plt.yticks(y_ticks, [f'{val:.2f}' for val in y_ticks])

            # Configurar el título y etiquetas
            plt.xlabel('Tipo de No conformidad', fontsize=12)
            plt.ylabel('Porcentaje de No conformidad', fontsize=12)
            plt.xticks(rotation=45, ha='right')

            # Ajustar el diseño para evitar recortes
            plt.legend(title=variable, bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.tight_layout()

            # Guardar la imagen del gráfico en un buffer
            buf = io.BytesIO()
            plt.savefig(buf, format="png")
            buf.seek(0)
            plt.close()

            # Codificar la imagen en base64 para mostrarla en el dashboard
            encoded_image = base64.b64encode(buf.read()).decode('utf-8')
            buf.close()

            # Retornar el gráfico como una imagen en el layout de Dash
            return html.Div([
                html.Img(
                    src=f'data:image/png;base64,{encoded_image}',
                    style={'width': '100%', 'height': 'auto'}
                )
            ])
        else:
            return html.Div("La variable seleccionada no está disponible en los datos.")

#Callback para el layaout dinamico
@app.callback(
    Output('layout-dinamico', 'children'),
    Input('tabs', 'value'),
    Input('upload-data-1', 'contents'),
    Input('upload-data-2', 'contents')
)
def generar_layout_dinamico(selected_tab, contents1, contents2):
    if selected_tab == 'model-tab' and contents1 and contents2:
      try:
          # Procesar archivos cargados
          content_type1, content_string1 = contents1.split(',')
          content_type2, content_string2 = contents2.split(',')
          decoded1 = base64.b64decode(content_string1)
          decoded2 = base64.b64decode(content_string2)

          df_informe = leer_archivo_seguro(decoded1)
          df_planta = leer_archivo_seguro(decoded2)

          # Combinar las bases de datos
          BD_fallas = pd.merge(
              df_planta, df_informe,
              left_on="ConsecPedido", right_on="Pedido",
              how="inner"
          )

          # Eliminar duplicados
          BD_fallas = BD_fallas.drop_duplicates(subset=["ConsecPedido", "NombProducto"])  # Eliminar duplicados por columnas clave

          # Crear un diccionario para corregir los nombres de las columnas

          correcciones = {
              'PaÃ­s': 'País',
              'Si escogiÃ³ otros, cuÃ¡l?': 'Si escogió otros, ¿cuál?',
              'Escoja el desgaste donde se presenta': 'Escoja el desgaste donde se presenta',
              'Porcentaje de elongaciÃ³n actual % (CADENA)': 'Porcentaje de elongación actual % (CADENA)',
              'Porcentaje de elongaciÃ³n mÃ¡x % (CADENA)': 'Porcentaje de elongación máx % (CADENA)',
              'ProyecciÃ³n en horas de operaciÃ³n restantes (H) (CADENA)': 'Proyección en horas de operación restantes (H) (CADENA)',
              'Medida sin desgaste en mm (BUJE)': 'Medida sin desgaste en mm (BUJE)',
              'Medida actual en mm1 (BUJE)': 'Medida actual en mm1 (BUJE)',
              'Porcentaje de desgaste actual % (BUJE)': 'Porcentaje de desgaste actual % (BUJE)',
              'Medida sin desgaste en mm1 (PASADOR)': 'Medida sin desgaste en mm1 (PASADOR)',
              'Medida actual en mm (PASADOR)': 'Medida actual en mm (PASADOR)',
              'Porcentaje de desgaste actual % (PASADOR)': 'Porcentaje de desgaste actual % (PASADOR)',
              'Medida sin desgaste en mm2 (RODILLO)': 'Medida sin desgaste en mm2 (RODILLO)',
              'Medida actual en mm2 (RODILLO)': 'Medida actual en mm2 (RODILLO)',
              'Porcentaje de desgaste actual % (RODILLO)': 'Porcentaje de desgaste actual % (RODILLO)',
              'Altura de la platina en mm': 'Altura de la platina en mm',
              'Medida actual de la platina en mm (PLATINA)': 'Medida actual de la platina en mm (PLATINA)',
              'Porcentaje de desgaste actual % (PLATINA)': 'Porcentaje de desgaste actual % (PLATINA)',
              'Paso en mm': 'Paso en mm',
              'Cantidad de pasos': 'Cantidad de pasos'
          }

          # Aplicar las correcciones al DataFrame
          BD_fallas.rename(columns=correcciones, inplace=True)

          # Corregir el nombre de los resultados de la variable 'Causa de desgaste'
          correcciones_causa = {
              'ElongaciÃ³n prematura;DesalineaciÃ³n;Otros': 'Elongación prematura;Desalineación;Otros',
              'ElongaciÃ³n prematura;CavitaciÃ³n seca': 'Elongación prematura;Cavitación seca',
              'DesalineaciÃ³n': 'Desalineación',
              'ElongaciÃ³n prematura;Otros': 'Elongación prematura;Otros',
              'CavitaciÃ³n seca': 'Cavitación seca',
              'Otros': 'Otros',
              'DesalineaciÃ³n;ElongaciÃ³n prematura;Otros;Falta de lubricaciÃ³n;Ruptura': 'Desalineación;Elongación prematura;Otros;Falta de lubricación;Ruptura',
              'Velocidad incorrecta;DesalineaciÃ³n': 'Velocidad incorrecta;Desalineación',
              'DesalineaciÃ³n;CavitaciÃ³n seca;Otros': 'Desalineación;Cavitación seca;Otros',
              'Medidas fuera de especificaciÃ³n': 'Medidas fuera de especificación',
              'Ruptura': 'Ruptura',
              'ElongaciÃ³n prematura': 'Elongación prematura',
              'Falta de lubricaciÃ³n': 'Falta de lubricación'
          }

          BD_fallas['Causa de desgaste'] = BD_fallas['Causa de desgaste'].replace(correcciones_causa)

          # Corregir los valores en la columna 'Material'
          BD_fallas['Material'] = BD_fallas['Material'].replace({
              'CAÃ\x91A': 'CAÑA',
              'CARBÃ\x93N GRANULADO': 'CARBÓN GRANULADO',
              pd.NA: 'DESCONOCIDO'
          })

          # Corregir los valores en la columna 'País'
          BD_fallas['País'] = BD_fallas['País'].replace({
              'PANAMÃ\x81': 'PANAMA'
          })

          # Cambio de formato de fecha
          BD_fallas['FechEstInicil'] = pd.to_datetime(BD_fallas['FechEstInicil'], errors='coerce')
          BD_fallas['Fecha de visita'] = pd.to_datetime(BD_fallas['Fecha de visita'], errors='coerce')

          # Crear una nueva columna 'Tiempo hasta falla'
          BD_fallas['Tiempo hasta falla (Dias)'] = (BD_fallas['Fecha de visita'] - BD_fallas['FechEstInicil']).dt.days

          # Rellenar los valores nulos con la cadena 'No Aplica'
          BD_fallas = BD_fallas.fillna('No Aplica')

          # Cambiar los nombres de las columnas
          BD_fallas.rename(columns={
              "Falla": "No conformidad",
              "Valor": "Porcentaje de no conformidad"
          }, inplace=True)


          # Función personalizada para revisar y reemplazar los valores de la columna 'NombProducto'
          def reemplazar_producto(nombre):
              if 'PTNA' in nombre:
                  return 'PLATINA'
              elif 'BUJE' in nombre:
                  return 'BUJE'
              elif 'CDNA' in nombre:
                  return 'CADENA'
              elif 'PASADOR' in nombre:
                  return 'PASADOR'
              elif 'RODILLO' in nombre:
                  return 'RODILLO'
              else:
                  return nombre

          BD_fallas['NombProducto'] = BD_fallas['NombProducto'].apply(reemplazar_producto)

          # Filtrar referencias de cadena con valores válidos
          referencias_validas = BD_fallas[
              pd.to_numeric(BD_fallas['Porcentaje de elongación máx % (CADENA)'], errors='coerce').notna()
          ]['Referencia cadena'].unique()

          # Variables y valores únicos (con referencias filtradas)
          variables = [
              'NombProducto', 'Empresa', 'Planta', 'País', 'Ciudad', 'Tipo de equipo',
              'Material', 'Causa de desgaste', 'Si escogió otros, ¿cuál?',
              'Escoja el desgaste donde se presenta', 'Tipo de cadena',
              'Referencia cadena', 'No conformidad', 'Tiempo hasta falla (Dias)'
          ]
          valores_unicos = {var: BD_fallas[var].unique().tolist() for var in variables}
          valores_unicos['Referencia cadena'] = referencias_validas  # Actualizar referencias válidas

          # Lista de las columnas que deseas eliminar
          columnas_a_eliminar = [
              'ConsecutivoOP', 'Producto','ConsecPedido', 'Cliente', 'OCCliente', 'SecAct', 'estado',
              'TipoConsecutivo', 'Pedido', 'Medida referencia de control desgaste nueva en mm (CADENA)',
              'Medida referencia control de desgaste actual en mm (CADENA)', 'Porcentaje de elongación actual % (CADENA)',
              'Porcentaje de elongación máx % (CADENA)', 'Proyección en horas de operación restantes (H) (CADENA)',
              'Medida sin desgaste en mm (BUJE)', 'Medida actual en mm1 (BUJE)', 'Porcentaje de desgaste actual % (BUJE)',
              'Medida sin desgaste en mm1 (PASADOR)', 'Medida actual en mm (PASADOR)',
              'Porcentaje de desgaste actual % (PASADOR)', 'Medida sin desgaste en mm2 (RODILLO)',
              'Medida actual en mm2 (RODILLO)', 'Porcentaje de desgaste actual % (RODILLO)',
              'Altura de la platina en mm', 'Medida actual de la platina en mm (PLATINA)',
              'Porcentaje de desgaste actual % (PLATINA)', 'Paso en mm', 'Cantidad de pasos', 'FechEstInicil', 'Fecha de visita',
              'Actividad','RecMAQ','RecHOM'
          ]

          # Generar un nuevo DataFrame eliminando las columnas especificadas
          BD_fallasD = BD_fallas.drop(columns=columnas_a_eliminar)


          # Convertir todos los valores a su valor absoluto en la columna 'Tiempo hasta falla (Dias)'
          BD_fallasD['Tiempo hasta falla (Dias)'] = BD_fallasD['Tiempo hasta falla (Dias)'].abs()

          # Remosión de outliers
          # Lista de valores a eliminar
          valores_a_eliminar = [18.69, 6.11, 9.48, 4.97, 4, 3.38, 3.25, 3.09]

          # Filtrar el DataFrame para eliminar filas con esos valores en 'Porcentaje de no conformidad'
          BD_fallasD = BD_fallasD[~BD_fallasD['Porcentaje de no conformidad'].isin(valores_a_eliminar)]



          # Filtra las filas donde 'NombProducto' contenga la palabra 'CADENA'
          BD_fallasD = BD_fallasD[BD_fallasD['NombProducto'].str.contains('CADENA', case=False, na=False)]


          # Cargar datos
          data = BD_fallasD

          # Separar variables predictoras y objetivo
          X = data.drop(columns=['Porcentaje de no conformidad'])
          y = data['Porcentaje de no conformidad']

          # Identificar columnas categóricas y numéricas
          categorical_cols = X.select_dtypes(include=['object']).columns
          numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

          # Preprocesamiento: One-Hot Encoding para categóricas, Escalado para numéricas
          preprocessor = ColumnTransformer(
              transformers=[
                  ('num', StandardScaler(), numerical_cols),
                  ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
              ]
          )

          # Selección de variables
          feature_selector = SelectKBest(score_func=f_regression, k='all')  # Cambiar 'all' por el número deseado de características

          # Definir modelos candidatos

          m_tree_reg = DecisionTreeRegressor(random_state=42)
          m_rf_reg = RandomForestRegressor(random_state=42)
          m_gbt_reg = GradientBoostingRegressor(random_state=42)

          models = {
              'Decision Tree': m_tree_reg,
              'Random Forest': m_rf_reg,
              'Gradient Boosting': m_gbt_reg
          }

          # Dividir datos en conjuntos de entrenamiento y prueba
          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          # Evaluar modelos
          results = []
          for name, model in models.items():
              pipeline = Pipeline(steps=[
                  ('preprocessor', preprocessor),
                  ('feature_selection', feature_selector),  # Selección de variables
                  ('model', model)
              ])
              pipeline.fit(X_train, y_train)

              # Predicciones
              y_pred = pipeline.predict(X_test)

              # Métricas
              mse = mean_squared_error(y_test, y_pred)
              mae = mean_absolute_error(y_test, y_pred)
              mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

              results.append((name, mse, mae, mape))

          # Crear un DataFrame con los resultados
          results_df = pd.DataFrame(results, columns=['Model', 'MSE', 'MAE', 'MAPE'])

          # Ordenar por MSE
          results_df = results_df.sort_values(by='MSE')

          # Mostrar el mejor modelo
          best_model = results_df.iloc[0]

          # Configuración de hiperparámetros para cada modelo
          param_grid = {
              'Decision Tree': {
                  'model__max_depth': [None, 10, 20, 30, 40],
                  'model__min_samples_split': [2, 5, 10],
                  'model__min_samples_leaf': [1, 2, 4]
              },
              'Random Forest': {
                  'model__n_estimators': [50, 100, 200],
                  'model__max_depth': [None, 10, 20, 30],
                  'model__min_samples_split': [2, 5, 10],
                  'model__min_samples_leaf': [1, 2, 4]
              },
              'Gradient Boosting': {
                  'model__n_estimators': [50, 100, 200],
                  'model__learning_rate': [0.01, 0.1, 0.2],
                  'model__max_depth': [3, 5, 7],
                  'model__min_samples_split': [2, 5, 10],
                  'model__min_samples_leaf': [1, 2, 4]
              }
          }

          # Obtener el modelo ganador desde el DataFrame de resultados
          best_model_name = results_df.iloc[0]['Model']  # Seleccionar el modelo con menor MSE (ya está ordenado en results_df)

          # Validar que el modelo existe en el diccionario
          if best_model_name not in models:
              raise ValueError(f"El modelo {best_model_name} no se encuentra en la lista de modelos disponibles.")

          best_model = models[best_model_name]

          # Configurar el pipeline con el modelo ganador
          pipeline = Pipeline(steps=[
              ('preprocessor', preprocessor),
              ('feature_selection', feature_selector),
              ('model', best_model)
          ])

          # Configurar el GridSearchCV
          grid_search = GridSearchCV(
              estimator=pipeline,
              param_grid=param_grid[best_model_name],
              scoring='neg_mean_squared_error',  # Usar MSE como métrica de optimización
              cv=5,  # Validación cruzada
              n_jobs=-1,  # Usar todos los núcleos disponibles
              verbose=2
          )

          # Ajustar el modelo con los datos de entrenamiento
          grid_search.fit(X_train, y_train)

          # Mejor modelo y sus hiperparámetros
          best_pipeline = grid_search.best_estimator_
          best_params = grid_search.best_params_

          # Evaluar el mejor modelo en el conjunto de prueba
          y_pred = best_pipeline.predict(X_test)
          mse = mean_squared_error(y_test, y_pred)
          mae = mean_absolute_error(y_test, y_pred)
          mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

          print("\nDesempeño del modelo ajustado en el conjunto de prueba:")
          print(f"MSE: {mse:.4f}")
          print(f"MAE: {mae:.4f}")
          print(f"MAPE: {mape:.2f}%")

          # Comparar todos los modelos y exportar el mejor
          all_results = []
          for model_name, model in models.items():
              pipeline = Pipeline(steps=[
                  ('preprocessor', preprocessor),
                  ('feature_selection', feature_selector),
                  ('model', model)
              ])
              pipeline.fit(X_train, y_train)
              y_pred = pipeline.predict(X_test)
              mse = mean_squared_error(y_test, y_pred)
              mae = mean_absolute_error(y_test, y_pred)
              mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
              all_results.append((model_name, mse, mae, mape, pipeline))

          # Seleccionar el mejor modelo por MSE
          best_overall_model = min(all_results, key=lambda x: x[1])

          print("\nEl mejor modelo final es:")
          print(f"Modelo: {best_overall_model[0]}, MSE: {best_overall_model[1]:.4f}, MAE: {best_overall_model[2]:.4f}, MAPE: {best_overall_model[3]:.2f}%")


          # Exportar el mejor modelo con sus metadatos
          model_metadata = {
              'pipeline': best_overall_model[4],  # El pipeline del modelo
              'model_name': best_model_name,      # Nombre del modelo
              'best_params': best_params,         # Hiperparámetros del GridSearch
              'mse': mse,                         # MSE en el conjunto de prueba
              'mae': mae,                         # MAE en el conjunto de prueba
              'mape': mape                        # MAPE en el conjunto de prueba
          }
          joblib.dump(model_metadata, 'best_model.pkl')

          # Cargar el modelo exportado junto con sus metadatos
          best_model_path = 'best_model.pkl'
          model_data = joblib.load(best_model_path)

          # Extraer detalles del modelo
          model_pipeline = model_data['pipeline']
          best_model_name = model_data.get('model_name', 'Modelo Desconocido')
          best_params = model_data.get('best_params', {})
          mse = model_data.get('mse', 'N/A')
          mae = model_data.get('mae', 'N/A')
          mape = model_data.get('mape', 'N/A')

          # Obtener el valor máximo de "Tiempo hasta falla (Dias)"
          max_tiempo_falla = BD_fallas['Tiempo hasta falla (Dias)'].max()
          min_tiempo_falla = abs(BD_fallas['Tiempo hasta falla (Dias)'].min())

          # Crear el layout dinámico
          layout_dinamico = html.Div([
              html.H3(
                  "Resultados del Modelo",
                  style={"font-family": "'Poppins', sans-serif"}
              ),
              html.P(
                  f"Mejor modelo seleccionado: {best_model_name}",
                  style={"font-family": "'Poppins', sans-serif"}
              ),
              html.P(
                  f"MSE: {mse:.2f}" if mse != 'N/A' else "MSE: No disponible",
                  style={"font-family": "'Poppins', sans-serif"}
              ),
              html.P(
                  f"MAE: {mae:.2f}" if mae != 'N/A' else "MAE: No disponible",
                  style={"font-family": "'Poppins', sans-serif"}
              ),
              html.P(
                  f"MAPE: {mape:.2f}%" if mape != 'N/A' else "MAPE: No disponible",
                  style={"font-family": "'Poppins', sans-serif"}
              ),
              html.Br(),

              # Interfaz para selección de variables
              html.Div([
                  html.Div([
                      html.Label(var, style={"font-family": "'Poppins', sans-serif"}),
                      dcc.Dropdown(
                          id=f'dropdown-{var}',
                          options=[{'label': val, 'value': val} for val in valores_unicos[var]],
                          value=valores_unicos[var][0]
                      )
                  ]) for var in variables if var not in ['Tiempo hasta falla (Dias)', 'No conformidad']
              ]),
              html.Br(),

              # Inputs para fechas
              html.Label("Fecha inicial:", style={"font-family": "'Poppins', sans-serif"}),
              dcc.Input(
                  id='input-fecha-inicial',
                  type='date',
                  placeholder='Ingrese la fecha inicial',
                  style={'margin-bottom': '10px', "font-family": "'Poppins', sans-serif"}
              ),
              html.Label("Fecha final:", style={"font-family": "'Poppins', sans-serif"}),
              dcc.Input(
                  id='input-fecha-final',
                  type='date',
                  placeholder='Ingrese la fecha final',
                  style={'margin-bottom': '20px', "font-family": "'Poppins', sans-serif"}
              ),

              # Campo para las horas actuales de servicio
              html.Label("Horas actuales de servicio (HAS):", style={"font-family": "'Poppins', sans-serif"}),
              dcc.Input(
                  id='input-has',
                  type='number',
                  placeholder='Ingrese las horas actuales de servicio',
                  style={'margin-bottom': '20px', "font-family": "'Poppins', sans-serif"}
              ),


              html.P(
                  f"Nota: Para la diferencia de fechas el minimo valor permitito es {min_tiempo_falla} y el maximo es {max_tiempo_falla} días.",
                  style={'color': 'red', 'font-weight': 'bold', 'margin-bottom': '20px', "font-family": "'Poppins', sans-serif"}
              ),


              # Outputs
              html.Div(id='output-prediccion', style={'font-size': '20px', 'font-weight': 'bold', "font-family": "'Poppins', sans-serif"}),
              html.Div(id='output-hps', style={'font-size': '20px', 'font-weight': 'bold', 'margin-top': '10px', "font-family": "'Poppins', sans-serif"})
          ], style={"font-family": "'Poppins', sans-serif"})

          return layout_dinamico

      except Exception as e:
          return html.Div(f"Error al generar el layout: {str(e)}")

# Callback para actualizar la predicción y calcular HPS
@app.callback(
    [Output('output-prediccion', 'children'),
     Output('output-hps', 'children')],
    [Input(f'dropdown-{var}', 'value') for var in [
        'NombProducto', 'Empresa', 'Planta', 'País', 'Ciudad', 'Tipo de equipo',
        'Material', 'Causa de desgaste', 'Si escogió otros, ¿cuál?',
        'Escoja el desgaste donde se presenta', 'Tipo de cadena',
        'Referencia cadena'
    ]] + [
        Input('input-fecha-inicial', 'value'),
        Input('input-fecha-final', 'value'),
        Input('input-has', 'value'),
        Input('upload-data-1', 'contents'),
        Input('upload-data-2', 'contents')
    ]
)
def actualizar_prediccion_y_hps(*valores):
    try:
        # Separar inputs
        fecha_inicial = valores[-5]
        fecha_final = valores[-4]
        has = valores[-3]
        contents1 = valores[-2]
        contents2 = valores[-1]
        valores = valores[:-5]  # Eliminar las fechas, HAS y archivos de los valores

                # Obtener la selección de NombProducto
        nomb_producto = valores[0]  # Primer valor corresponde a NombProducto

        # Asignar automáticamente el valor de "No conformidad"
        if nomb_producto == "CADENA":
            no_conformidad = "Elongación"
        else:
            no_conformidad = "Desgaste"

        # Agregar "No conformidad" al input del modelo
        entrada = pd.DataFrame([{var: val for var, val in zip(
            ['NombProducto', 'Empresa', 'Planta', 'País', 'Ciudad', 'Tipo de equipo',
             'Material', 'Causa de desgaste', 'Si escogió otros, ¿cuál?',
             'Escoja el desgaste donde se presenta', 'Tipo de cadena',
             'Referencia cadena'], valores
        )}])
        entrada['No conformidad'] = no_conformidad

        if contents1 and contents2:
            # Procesar archivos cargados
            content_type1, content_string1 = contents1.split(',')
            content_type2, content_string2 = contents2.split(',')
            decoded1 = base64.b64decode(content_string1)
            decoded2 = base64.b64decode(content_string2)

            # Leer y combinar las bases de datos
            df_informe = leer_archivo_seguro(decoded1)
            df_planta = leer_archivo_seguro(decoded2)
            BD_fallas = pd.merge(
                df_planta, df_informe,
                left_on="ConsecPedido", right_on="Pedido",
                how="inner"
            )

            # Eliminar duplicados
            BD_fallas = BD_fallas.drop_duplicates(subset=["ConsecPedido", "NombProducto"])  # Eliminar duplicados por columnas clave

            # Crear un diccionario para corregir los nombres de las columnas

            correcciones = {
                'PaÃ­s': 'País',
                'Si escogiÃ³ otros, cuÃ¡l?': 'Si escogió otros, ¿cuál?',
                'Escoja el desgaste donde se presenta': 'Escoja el desgaste donde se presenta',
                'Porcentaje de elongaciÃ³n actual % (CADENA)': 'Porcentaje de elongación actual % (CADENA)',
                'Porcentaje de elongaciÃ³n mÃ¡x % (CADENA)': 'Porcentaje de elongación máx % (CADENA)',
                'ProyecciÃ³n en horas de operaciÃ³n restantes (H) (CADENA)': 'Proyección en horas de operación restantes (H) (CADENA)',
                'Medida sin desgaste en mm (BUJE)': 'Medida sin desgaste en mm (BUJE)',
                'Medida actual en mm1 (BUJE)': 'Medida actual en mm1 (BUJE)',
                'Porcentaje de desgaste actual % (BUJE)': 'Porcentaje de desgaste actual % (BUJE)',
                'Medida sin desgaste en mm1 (PASADOR)': 'Medida sin desgaste en mm1 (PASADOR)',
                'Medida actual en mm (PASADOR)': 'Medida actual en mm (PASADOR)',
                'Porcentaje de desgaste actual % (PASADOR)': 'Porcentaje de desgaste actual % (PASADOR)',
                'Medida sin desgaste en mm2 (RODILLO)': 'Medida sin desgaste en mm2 (RODILLO)',
                'Medida actual en mm2 (RODILLO)': 'Medida actual en mm2 (RODILLO)',
                'Porcentaje de desgaste actual % (RODILLO)': 'Porcentaje de desgaste actual % (RODILLO)',
                'Altura de la platina en mm': 'Altura de la platina en mm',
                'Medida actual de la platina en mm (PLATINA)': 'Medida actual de la platina en mm (PLATINA)',
                'Porcentaje de desgaste actual % (PLATINA)': 'Porcentaje de desgaste actual % (PLATINA)',
                'Paso en mm': 'Paso en mm',
                'Cantidad de pasos': 'Cantidad de pasos'
            }

            # Aplicar las correcciones al DataFrame
            BD_fallas.rename(columns=correcciones, inplace=True)

            # Corregir el nombre de los resultados de la variable 'Causa de desgaste'
            correcciones_causa = {
                'ElongaciÃ³n prematura;DesalineaciÃ³n;Otros': 'Elongación prematura;Desalineación;Otros',
                'ElongaciÃ³n prematura;CavitaciÃ³n seca': 'Elongación prematura;Cavitación seca',
                'DesalineaciÃ³n': 'Desalineación',
                'ElongaciÃ³n prematura;Otros': 'Elongación prematura;Otros',
                'CavitaciÃ³n seca': 'Cavitación seca',
                'Otros': 'Otros',
                'DesalineaciÃ³n;ElongaciÃ³n prematura;Otros;Falta de lubricaciÃ³n;Ruptura': 'Desalineación;Elongación prematura;Otros;Falta de lubricación;Ruptura',
                'Velocidad incorrecta;DesalineaciÃ³n': 'Velocidad incorrecta;Desalineación',
                'DesalineaciÃ³n;CavitaciÃ³n seca;Otros': 'Desalineación;Cavitación seca;Otros',
                'Medidas fuera de especificaciÃ³n': 'Medidas fuera de especificación',
                'Ruptura': 'Ruptura',
                'ElongaciÃ³n prematura': 'Elongación prematura',
                'Falta de lubricaciÃ³n': 'Falta de lubricación'
            }

            BD_fallas['Causa de desgaste'] = BD_fallas['Causa de desgaste'].replace(correcciones_causa)

            # Corregir los valores en la columna 'Material'
            BD_fallas['Material'] = BD_fallas['Material'].replace({
                'CAÃ\x91A': 'CAÑA',
                'CARBÃ\x93N GRANULADO': 'CARBÓN GRANULADO',
                pd.NA: 'DESCONOCIDO'
            })

            # Corregir los valores en la columna 'País'
            BD_fallas['País'] = BD_fallas['País'].replace({
                'PANAMÃ\x81': 'PANAMA'
            })

            # Cambio de formato de fecha
            BD_fallas['FechEstInicil'] = pd.to_datetime(BD_fallas['FechEstInicil'], errors='coerce')
            BD_fallas['Fecha de visita'] = pd.to_datetime(BD_fallas['Fecha de visita'], errors='coerce')

            # Crear una nueva columna 'Tiempo hasta falla'
            BD_fallas['Tiempo hasta falla (Dias)'] = (BD_fallas['Fecha de visita'] - BD_fallas['FechEstInicil']).dt.days

            # Rellenar los valores nulos con la cadena 'No Aplica'
            BD_fallas = BD_fallas.fillna('No Aplica')

            # Cambiar los nombres de las columnas
            BD_fallas.rename(columns={
                "Falla": "No conformidad",
                "Valor": "Porcentaje de no conformidad"
            }, inplace=True)

            # Función personalizada para revisar y reemplazar los valores de la columna 'NombProducto'
            def reemplazar_producto(nombre):
                if 'PTNA' in nombre:
                    return 'PLATINA'
                elif 'BUJE' in nombre:
                    return 'BUJE'
                elif 'CDNA' in nombre:
                    return 'CADENA'
                elif 'PASADOR' in nombre:
                    return 'PASADOR'
                elif 'RODILLO' in nombre:
                    return 'RODILLO'
                else:
                    return nombre

            BD_fallas['NombProducto'] = BD_fallas['NombProducto'].apply(reemplazar_producto)


            # Filtrar referencias de cadena con valores válidos
            referencias_validas = BD_fallas[
                pd.to_numeric(BD_fallas['Porcentaje de elongación máx % (CADENA)'], errors='coerce').notna()
            ]['Referencia cadena'].unique()

            # Variables y valores únicos (con referencias filtradas)
            variables = [
                'NombProducto', 'Empresa', 'Planta', 'País', 'Ciudad', 'Tipo de equipo',
                'Material', 'Causa de desgaste', 'Si escogió otros, ¿cuál?',
                'Escoja el desgaste donde se presenta', 'Tipo de cadena',
                'Referencia cadena', 'No conformidad', 'Tiempo hasta falla (Dias)'
            ]
            valores_unicos = {var: BD_fallas[var].unique().tolist() for var in variables}
            valores_unicos['Referencia cadena'] = referencias_validas  # Actualizar referencias válidas

            # Validar fechas
            if not fecha_inicial or not fecha_final:
                return "Por favor, ingrese ambas fechas.", ""

            fecha_inicial = pd.to_datetime(fecha_inicial)
            fecha_final = pd.to_datetime(fecha_final)
            diferencia_dias = (fecha_final - fecha_inicial).days

            if diferencia_dias <= 0:
                return "La fecha final debe ser posterior a la fecha inicial.", ""

            # Buscar el valor más cercano por encima en "Tiempo hasta falla (Dias)"
            valores_tiempo = BD_fallas['Tiempo hasta falla (Dias)'].unique()
            valores_tiempo.sort()
            tiempo_seleccionado = next((v for v in valores_tiempo if v >= diferencia_dias), None)

            if tiempo_seleccionado is None:
                return f"No se encontró un valor válido para {diferencia_dias} días en 'Tiempo hasta falla (Dias)'.", ""

            entrada['Tiempo hasta falla (Dias)'] = tiempo_seleccionado

            # Cargar el modelo
            best_model_path = 'best_model.pkl'
            model_data = joblib.load(best_model_path)
            model_pipeline = model_data.get('pipeline')

            # Realizar la predicción
            prediccion = model_pipeline.predict(entrada)[0]
            prediccion_texto = f"Porcentaje de No Conformidad: {prediccion:.2f}%"

            # Buscar el porcentaje de elongación máximo en BD_fallas
            referencia_cadena = entrada['Referencia cadena'][0]
            porcentaje_elongacion = BD_fallas.loc[
                BD_fallas['Referencia cadena'] == referencia_cadena,
                'Porcentaje de elongación máx % (CADENA)'
            ].values

            if len(porcentaje_elongacion) == 0:
                return prediccion_texto, "Referencia de cadena no encontrada en la base de datos."

            porcentaje_elongacion = float(porcentaje_elongacion[0])

            # Verificar que HAS sea válido
            if has is None or has <= 0:
                return prediccion_texto, "Por favor, ingrese un valor válido para las horas actuales de servicio (HAS)."

            # Calcular las horas proyectadas de servicio (HPS)
            hps = (porcentaje_elongacion / prediccion) * has
            hps_texto = f"Horas Proyectadas de Servicio (HPS): {hps:.2f}"

            return prediccion_texto, hps_texto

        else:
            return "Por favor, cargue las bases de datos.", ""

    except Exception as e:
        # Manejo de errores
        return f"Error en la predicción: {str(e)}", ""


# Ejecución de la aplicación en un servidor local
if __name__ == '__main__':
    app.run_server(debug=False, host='0.0.0.0', port=8053)  # Configuración del servidor


<IPython.core.display.Javascript object>